<a href="https://colab.research.google.com/github/talhaanwarch/Facial-Expression-Recognition-Using-VGGface/blob/master/en_ak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-dc143809-12a5-6b17-cbf4-293fadaab148)


#install pacakges

In [2]:
%%capture
!python3 -m pip install autokeras

# download data

In [3]:
import os
if os.path.isdir('/content/Profiling-Hate-Speech-Spreaders-on-Twitter') is False:
    !git clone https://github.com/talhaanwarch/Profiling-Hate-Speech-Spreaders-on-Twitter.git

Cloning into 'Profiling-Hate-Speech-Spreaders-on-Twitter'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 28 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [4]:
%%capture 
if os.path.isdir('train') is False:
  !unzip /content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/train.zip
if os.path.isdir('test') is False:
  !unzip /content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/test.zip

# prepare data

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df=pd.read_csv('train/en/truth.txt',sep=':::',header=None,engine='python')
df.columns=['id','label']

In [6]:
df.label.value_counts()

1    100
0    100
Name: label, dtype: int64

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
def preprocessing(text):
    text=text.str.replace('\d+', '')
    text=text.str.replace('RT','')
    text=text.str.replace('#USER#','')
    text=text.str.replace('#URL#','')
    text= text.str.lower()
    text = text.str.replace('[^\w\s]','')
    text = text.apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
    text = text.apply(lambda x: [item for item in x if item not in stop])
    text = text.apply(lambda x : " ".join(x))
    return text

In [9]:
df.id='train/en/'+df.id.astype(str)+'.xml'
df.head()

,id,label
0,train/en/639b8e5e6a527d494c85d8f5704b1a01.xml,0
1,train/en/f2b1fc84c500c38a93522efbd422b559.xml,0
2,train/en/10b2d013382e1fb3c9414ea28329f258.xml,0
3,train/en/26644d1348fc1122e8c5ef45d6bc84fa.xml,0
4,train/en/4d4c5dcbfe38d0d33a0d1b1419952ca8.xml,0


In [10]:
len(df)

200

In [11]:
import xml.etree.ElementTree as ET
def reader(df,ground=True):
  data=[]
  for x in df.iterrows():
      
      tree = ET.parse(x[1].id)
      root = tree.getroot()
      text=[x.text for x in root[0]]
      if ground:
        label=[x[1].label]*len(text)
        data.append(pd.DataFrame(zip(text,label),columns=['text','label']))
      else:
        data.append(pd.DataFrame(text,columns=['text']))
  return data

In [12]:
data=reader(df)

In [13]:
data[0].head()

,text,label
0,RT #USER#: Funny how “15 days to slow the spre...,0
1,RT #USER#: Why did Minneapolis just give Georg...,0
2,"RT #USER#: To be fair, he has done a lot of un...",0
3,RT #USER#: President Trump got us the #HASHTAG...,0
4,RT #USER#: Is the case against former officer ...,0


In [14]:
from sklearn.model_selection import train_test_split
trainx,valx=train_test_split(data,test_size=0.1,)
trainx,testx=train_test_split(trainx,test_size=0.1)

In [15]:
train=pd.concat(trainx)
val=pd.concat(valx)
test=pd.concat(testx)

In [16]:
train.label.value_counts()

1    16200
0    16200
Name: label, dtype: int64

In [17]:
val.label.value_counts()

0    2200
1    1800
Name: label, dtype: int64

In [18]:
test.label.value_counts()

1    2000
0    1600
Name: label, dtype: int64

In [19]:
train.text=preprocessing(train.text)
val.text=preprocessing(val.text)
test.text=preprocessing(test.text)

In [20]:
len(train),len(val),len(test)

(32400, 4000, 3600)

In [21]:
train = train.sample(frac=1).reset_index(drop=True)
val = val.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)


In [22]:
train_text=train.text.values
train_label=train.label.values
val_text=val.text.values
val_label=val.label.values
test_text=test.text.values
test_label=test.label.values

# classify data

In [23]:
import autokeras as ak
clf = ak.TextClassifier() 

In [ ]:
clf.fit(train_text,train_label,epochs=100,validation_data=(val_text, val_label))


Trial 5 Complete [00h 04m 25s]
val_loss: 0.6885523796081543

Best val_loss So Far: 0.6885523796081543
Total elapsed time: 00h 48m 18s

Search: Running Trial #6

Hyperparameter    |Value             |Best Value So Far 
text_block_1/bl...|transformer       |transformer       
classification_...|0.25              |0                 
optimizer         |adam              |adam              
learning_rate     |0.001             |0.001             
text_block_1/ma...|20000             |20000             
text_block_1/te...|200               |200               
text_block_1/tr...|none              |none              
text_block_1/tr...|32                |32                
text_block_1/tr...|2                 |2                 
text_block_1/tr...|32                |32                
text_block_1/tr...|0.25              |0.25              
text_block_1/sp...|global_max        |global_max        
text_block_1/de...|False             |False             
text_block_1/de...|1                 |1  

In [ ]:
val_score = clf.evaluate(val_text, val_label)

In [ ]:
test_score = clf.evaluate(test_text,test_label)

In [ ]:
from sklearn.metrics import classification_report
pred=clf.predict(val_text)
print(classification_report(val_label,pred))

In [ ]:
pred=clf.predict(test_text)
print(classification_report(test_label,pred))

# validation

In [ ]:
preds,trues=[],[]
for i in valx:
  pred=list(clf.predict(i.text.values,verbose=0))
  preds.append(max(pred,key=pred.count))
  trues.append(int(np.mean(i.label)))
print(classification_report(trues,preds))  

In [ ]:
preds,trues=[],[]
for i in testx:
  pred=list(clf.predict(i.text.values,verbose=0))
  preds.append(max(pred,key=pred.count))
  trues.append(int(np.mean(i.label)))
print(classification_report(trues,preds))  

# test evaluation

In [ ]:
from glob import glob
test_path=pd.DataFrame(glob('test/en/*.xml'),columns=['id'])
test_path.head()

In [ ]:
test=reader(test_path,ground=False)

In [ ]:
test[0].head()

In [ ]:
final_preds=[]
for i in range(len(test)):
  test[i].text=preprocessing(test[i].text)
  pred=list(clf.predict(test[i].text.values,verbose=0))
  final_preds.append(int(max(pred,key=pred.count)))


In [ ]:
len(final_preds),len(test_path)

In [ ]:
ids=test_path.id.apply(lambda x:x.split('/')[-1].split('.')[0])

In [ ]:
lang=['en']*len(test_path)

In [ ]:
final=pd.DataFrame(zip(ids,lang,final_preds),columns=['author-id','lang','type'])
final.head()

In [ ]:
final.to_csv('en_ag.csv',index=False)